# Cogneeを使った永続メモリを持つAIエージェントの構築

このノートブックでは、[**cognee**](https://www.cognee.ai/) を使用して、高度なメモリ機能を備えたインテリジェントなAIエージェントを構築する方法を示します。[cognee]は、ナレッジグラフ、セマンティック検索、セッション管理を組み合わせて、コンテキスト認識型のAIシステムを作成するオープンソースのAIメモリです。

## 🎯 学習目標

このチュートリアルの終わりまでに、以下のことを理解できるようになります：
- **埋め込みを活用したナレッジグラフの構築**: 非構造化テキストを構造化されたクエリ可能な知識に変換する
- **セッションメモリの実装**: 自動的にコンテキストを保持するマルチターンの会話を作成する
- **会話の永続化**: 重要なやり取りを長期記憶に保存し、将来参照できるようにする
- **自然言語でのクエリ**: 新しい会話で過去のコンテキストを活用する
- **メモリの可視化**: エージェントのナレッジグラフ内の関係を探索する


## 🏗️ 作成するもの

このチュートリアルでは、以下の機能を持つ**コーディングアシスタント**を作成します。特徴として、持続的な記憶を備えています。

### 1. **ナレッジベースの構築**
   - 開発者のプロフィールや専門知識情報を取り込む
   - Pythonのプログラミング原則やベストプラクティスを処理する
   - 開発者とAIアシスタント間の過去の会話を保存する

### 2. **セッション対応の会話**
   - 同じセッション内で複数の質問にわたる文脈を維持する
   - 各質問と回答のペアを自動的にキャッシュして効率的に取得する
   - 会話履歴に基づいて一貫性のある文脈的な応答を提供する

### 3. **長期記憶**
   - 重要な会話を長期記憶に保存する
   - ナレッジベースや過去のセッションから関連する記憶を取得し、新しいやり取りに活用する
   - 時間とともに成長するナレッジベースを構築する

### 4. **インテリジェントな記憶検索**
   - グラフ対応のセマンティック検索を使用して、保存されたすべての知識から関連情報を見つける
   - データのサブグループ（開発者情報 vs. 原則）で検索をフィルタリングする
   - 複数のデータソースを組み合わせて包括的な回答を提供する


## 📋 前提条件とセットアップ

### システム要件

開始する前に、以下を確認してください：

1. **Python環境**
   - Python 3.9以上
   - 仮想環境（推奨）

2. **Redisキャッシュ**（セッション管理に必要）
   - ローカルRedis: `docker run -d -p 6379:6379 redis`
   - または、マネージドRedisサービスを使用

3. **LLM APIアクセス**
   - OpenAI APIキーまたは他のプロバイダー（[ドキュメント](https://docs.cognee.ai/setup-configuration/llm-providers)を参照）

4. **データベース構成**
   - デフォルトでは構成不要。Cogneeはファイルベースのデータベース（LanceDBとKuzu）を使用
   - オプションで、Azure AI Searchをベクトルストアとして設定可能（[ドキュメント](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch)を参照）

### 環境設定

プロジェクトディレクトリに `.env` ファイルを作成し、以下の変数を記述してください：

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Cogneeのメモリアーキテクチャを理解する

### Cogneeの仕組み

Cogneeは、単純なキーと値のストレージを超えた高度なメモリシステムを提供します:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### 主な構成要素:

1. **ナレッジグラフ**: エンティティ、関係、意味的な接続を保存
2. **ベクトル埋め込み**: 保存されたすべての情報に対する意味検索を可能にする
3. **セッションキャッシュ**: セッション内およびセッション間で会話の文脈を保持
4. **NodeSets**: データを論理的なカテゴリに整理し、ターゲットを絞った取得を可能にする

### このチュートリアルでのメモリタイプ:

- **永続メモリ**: ナレッジグラフ内の長期保存
- **セッションメモリ**: Redisキャッシュ内の一時的な会話文脈
- **セマンティックメモリ**: すべてのデータにわたるベースの類似性検索


## 📦 必要なパッケージをインストール

セッション管理のためにRedisサポート付きのCogneeをインストールします:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 環境の初期化とライブラリの読み込み

以下を確認してください:
1. Redisが稼働していること（例: Dockerを使用する場合 `docker run -d -p 6379:6379 redis`）
2. 環境変数がキャッシュモジュールをインポートする前に設定されていること
3. 必要に応じてカーネルを再起動し、セルを順番に実行すること

以下のセルでは以下を行います:
1. `.env`から環境変数を読み込む
2. CogneeをLLM設定で構成する
3. セッション管理のためのキャッシュを有効化する
4. すべてのコンポーネントが正しく接続されていることを検証する


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 ストレージディレクトリの設定

Cogneeはその操作のために2つの異なるディレクトリを使用します：
- **データルート**: 取り込まれたドキュメントや処理済みデータを保存
- **システムルート**: ナレッジグラフデータベースとシステムメタデータを含む

このチュートリアルでは、以下のように分離されたディレクトリを作成します：


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 メモリ状態のリセット

メモリシステムを構築する前に、まずは新しい状態から始めるようにしましょう。

> 💡 **ヒント**: このノートブックを後で使用する際に、以前の実行からの既存のメモリを保持したい場合は、このステップをスキップすることができます。


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 パート1: ナレッジベースの構築

### 開発者アシスタントのためのデータソース

包括的なナレッジベースを作成するために、以下の3種類のデータを取り込みます:

1. **開発者プロフィール**: 個人の専門知識や技術的な背景  
2. **Pythonのベストプラクティス**: Pythonの哲学（The Zen of Python）と実践的なガイドライン  
3. **過去の会話履歴**: 開発者とAIアシスタント間の過去のQ&Aセッション  

この多様なデータにより、エージェントは以下を実現できます:
- ユーザーの技術的な背景を理解する  
- 推奨事項にベストプラクティスを適用する  
- 過去の成功したやり取りから学ぶ  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 データをナレッジグラフに変換

ここでは、生のテキストを構造化された記憶に変換します。このプロセスでは以下を行います：

1. **データをNodeSetsに追加**: 情報を論理的なカテゴリに整理
   - `developer_data`: 開発者のプロフィールと会話
   - `principles_data`: Pythonのベストプラクティスとガイドライン

2. **Cognify Pipelineを実行**: エンティティや関係を抽出し、埋め込みを作成
   - 重要な概念を特定
   - 関連情報間のセマンティックなつながりを作成
   - ベクトル埋め込みを生成

LLMがテキストを処理し、グラフ構造を構築するため、少し時間がかかる場合があります：


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 ナレッジグラフの可視化

ナレッジグラフの構造を見てみましょう。この可視化では以下を示しています：
- **ノード**: テキストから抽出されたエンティティ（概念、技術、人など）
- **エッジ**: エンティティ間の関係や接続
- **クラスター**: 意味的な類似性によってグループ化された関連する概念

生成されたHTMLファイルをブラウザで開いて、グラフをインタラクティブに探索してください:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Memifyで記憶を強化

`memify()`関数は、ナレッジグラフを分析し、データに関する知的なルールを生成します。このプロセスでは以下を行います：
- パターンやベストプラクティスを特定
- コンテンツに基づいた実行可能なガイドラインを作成
- 異なる知識領域間の関係を確立

これらのルールは、エージェントが質問に答える際に、より情報に基づいた意思決定を行うのに役立ちます。2回目のビジュアライゼーションをキャプチャすることで、グラフが強化された後にどのように密度が増したかを比較できます。


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 パート2: インテリジェントなメモリ検索

### デモンストレーション1: 複数ドキュメントの知識統合

知識グラフが構築されたので、Cogneeが複数の情報源から情報を組み合わせて複雑な質問に答える方法をテストしてみましょう。

最初のクエリでは以下を示します:
- **セマンティック理解**: 明示的に言及されていなくても関連する概念を見つける
- **クロスリファレンス**: 開発者のプロフィールとPythonの原則を組み合わせる
- **文脈的推論**: ベストプラクティスを特定の実装に適用する

### デモンストレーション2: NodeSetsを使ったフィルタ検索

2つ目のクエリでは、知識グラフの特定のサブセットをターゲットにする方法を示します:
- `node_name`パラメータを使用して`principles_data`内のみを検索
- 特定の知識分野から焦点を絞った回答を提供
- 分野特化の情報が必要な場合に便利


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 パート3: セッション管理のセットアップ

### 会話メモリの有効化

セッション管理は、複数のやり取りにおいてコンテキストを維持するために重要です。ここでは以下を行います:

1. **ユーザーコンテキストの初期化**: セッション追跡のためにユーザープロファイルを作成または取得します
2. **キャッシュエンジンの設定**: 会話履歴を保存するためにRedisに接続します
3. **セッション変数の有効化**: クエリ間で持続するコンテキスト変数を設定します

> ⚠️ **重要**: Redisが稼働しており、環境で`CACHING=true`が設定されている必要があります


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ ヘルパー関数: セッション履歴の表示

このユーティリティ関数は、Redisに保存されている会話履歴を確認するためのものです。以下の用途に役立ちます:
- セッション管理のデバッグ
- 会話がキャッシュされていることの確認
- エージェントが利用可能なコンテキストの把握


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## セッション1: 非同期サポートラボ — 最初の質問

`async-support-lab`セッションを開始し、大規模なウェブスクレイパー向けのテレメトリ対応のasyncioパターンについて質問してください。グラフはすでにasyncio、aiohttp、モニタリングの実践について知っているので、回答は以前の会話を反映しつつ、新しい質問に合わせた内容になるべきです。


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## セッション1のメモリを最初のやり取り後に確認する

最初の質問の直後に `show_history(session_1)` を実行すると、Cogneeがプロンプトと応答の両方をRedisに書き込んだことが確認できます。同時実行ガイダンスを含む1つのエントリが表示されるはずです。


In [ ]:
await show_history(session_1)

## セッション1: データモデルのフォローアップ

次に、「dataclassesとPydanticのどちらを選ぶべきかはいつか？」という質問をします。同じセッションIDを使用します。CogneeはPythonの原則と以前のFastAPIの会話を組み合わせて、文脈が名前付きセッション内で引き継がれることを示しながら、詳細なアドバイスを提供する必要があります。


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## セッション1の履歴に両方のターンが含まれていることを確認

もう一度 `show_history(session_1)` を呼び出すと、2つのQ&Aエントリが表示されるはずです。これはMem0ラボの「メモリ再生」ステップと一致し、追加のターンが同じトランスクリプトを拡張することを証明します。


In [ ]:
await show_history(session_1)

## セッション 2: デザインレビューのスレッド — 新しいセッション

スレッド間の分離を示すために、`design-review-session` を起動し、インシデントレビューのためのログガイダンスを求めます。基礎となる知識ベースは同じですが、新しいセッションIDによってトランスクリプトが分離されます。


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## レビューセッション2の履歴

`show_history(session_2)` は、デザインレビューのプロンプト/レスポンスペアのみをリストする必要があります。セッション1と比較して、Cogneeが共有ナレッジグラフを再利用しながら、独立したトランスクリプトをどのように保持しているかを強調してください。


In [ ]:
await show_history(session_2)

## 概要

おめでとうございます！あなたのコーディングアシスタントに、Cogneeによる本格的な長期記憶レイヤーを追加しました。

このチュートリアルでは、開発者向けの生データ（コード、ドキュメント、チャット）をグラフ＋ベクトルメモリに変換し、エージェントが検索、推論、そして継続的に改善できるようにする方法を学びました。

学んだこと

1. **生テキストからAIメモリへ**: Cogneeが非構造化データを取り込み、ベクトル＋知識グラフアーキテクチャを使用してインテリジェントで検索可能なメモリに変換する方法。

2. **memifyによるグラフの強化**: 基本的なグラフ作成を超えて、memifyを使用して既存のグラフに派生した事実やより豊かな関係を追加する方法。

3. **複数の検索戦略**: エージェントのニーズに応じて、異なる検索タイプ（グラフ対応のQ&A、RAGスタイルの補完、インサイト、生データのチャンク、コード検索など）を使用してメモリをクエリする方法。

4. **視覚的な探索**: グラフの可視化やCognee UIを使用して、Cogneeが構築した内容を検査・デバッグし、知識がどのように構造化されているかを実際に確認する方法。

5. **セッション対応のメモリ**: セッションごとのコンテキストを永続的なセマンティックメモリと組み合わせることで、エージェントがユーザー間で情報を漏らすことなく、複数回の実行にわたって記憶できるようにする方法。


## 重要なポイント
1. 埋め込みを活用した知識グラフとしてのメモリ

    - **構造化された理解**: Cogneeはベクトルストアとグラフストアを組み合わせることで、データを意味で検索可能にし、関係性で接続します。Cogneeはデフォルトでファイルベースのデータベースを使用します（ベクトルにはLanceDB、グラフデータベースにはKuzu）。

    - **関係性を考慮した検索**: 回答は「類似したテキスト」だけでなく、エンティティ間の関係性に基づいて導き出されます。

    - **進化するメモリ**: メモリ層は進化し、成長し、1つの接続されたグラフとして常にクエリ可能な状態を維持します。

2. 検索と推論モード
    - **ハイブリッド検索**: 検索はベクトル類似性、グラフ構造、LLM推論を組み合わせ、生のチャンク検索からグラフを考慮した質問応答まで対応します。

    - **目的に応じたモード選択**: 自然言語での回答が必要な場合は補完型モードを使用し、エージェントが生のコンテキストを必要とする場合や独自の推論を行う場合はチャンク/要約/グラフモードを使用します。

3. 個別化されたセッション対応エージェント
    - **セッションコンテキスト + 長期メモリ**: Cogneeは短期的な「スレッド」コンテキストを、長期的なユーザーや組織レベルのメモリから分離して保持します。

## 実際の応用例

1. **特定分野向けAIエージェント**

    このノートブックのパターンを使用して、Cogneeを検索と推論のコアとして活用する分野特化型のコパイロットを構築できます:

- **開発者向けコパイロット**: コードレビュー、インシデント分析、アーキテクチャ支援を行い、コード、API、設計文書、チケットを1つのメモリグラフとして横断します。

- **顧客対応コパイロット**: 製品文書、FAQ、CRMノート、過去のチケットをグラフ対応検索で引き出し、引用付きの回答を提供します。

- **内部専門家向けコパイロット**: ポリシー、法務、セキュリティ支援を行い、孤立したPDFではなく、相互に関連するルール、ガイドライン、過去の決定を基に推論します。

    CogneeはAIエージェントのための永続的で正確なメモリとして明確に位置付けられており、エージェントの背後にある生きた知識グラフを提供し、ベクトルストアやカスタムグラフコードのアドホックな組み合わせを置き換えます。

2. **データサイロを統一して1つのメモリに**

    同じアプローチで、分散したソースを統一したメモリ層に構築することも可能です:

- **サイロから1つのグラフへ**: 構造化データ（例: データベース）や非構造化データ（例: 文書、チャット）を埋め込みに基づいた単一のグラフに取り込み、各システムごとのインデックスを分離する代わりに統合します。

- **引用付きのクロスソース推論**: ログ、メトリクス、文書をグラフを介して「結合」し、すべてにわたる多段階推論を実行しながら、根拠のある回答を返します。

- **知識ハブ**: 銀行や教育のような分野では、CogneeはすでにPDF、内部システム、アプリデータを統一し、ベクトルを活用した知識グラフを構築しており、エージェントが正確で引用付きのコンテキストで質問に答えることができます。

## 次のステップ

コアメモリループを実装しました。以下は独自に試せる自然な拡張案です（詳細は[Cogneeのドキュメント](https://docs.cognee.ai/)をご覧ください）:

1. **時間認識の実験**: 時間認識をオンにして、テキストからイベントやタイムスタンプを抽出します。

2. **オントロジー駆動型推論の導入**: 自分の分野に合わせたOWLオントロジーを定義します。Cogneeのオントロジーサポートを活用して、抽出されたエンティティや関係をそのスキーマに基づいて整理し、グラフの品質や分野特化型の回答を向上させます。

3. **フィードバックループの追加**: 実際のユーザーフィードバックからCogneeがグラフのエッジウェイトを調整できるようにし、検索が静的ではなく時間とともに改善されるようにします。

4. **個別化とセッション動作の調整**: ユーザーID、テナント、データセットを使用して、共有メモリエンジン上で各個人やチームに独自のビューを提供します。

5. **より複雑なエージェントへの拡張**: Cogneeをエージェントフレームワークに接続し、同じメモリ層を共有するマルチエージェントシステムを構築します。*Microsoft Agent Framework x Cogneeプラグインが近日公開予定です。*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解について、当社は責任を負いません。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
